In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import json
from math import log10
from collections import Counter
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit, GridSearchCV
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/comp90051-2024s1-project-1/test_data.json
/kaggle/input/comp90051-2024s1-project-1/sample.csv
/kaggle/input/comp90051-2024s1-project-1/domain2_train_data.json
/kaggle/input/comp90051-2024s1-project-1/domain1_train_data.json


In [2]:
domain1_path = r"/kaggle/input/comp90051-2024s1-project-1/domain1_train_data.json" 
domain2_path = r"/kaggle/input/comp90051-2024s1-project-1/domain2_train_data.json"
domain_official_path = r"/kaggle/input/comp90051-2024s1-project-1/test_data.json"

def read_file(file_path):
    f = open(file_path)

    data = []
    for line in f:
        data.append(json.loads(line))
        
    return data
        
data1 = read_file(domain1_path)
data2 = read_file(domain2_path)
data_official = read_file(domain_official_path)

X1 = [data["text"] for data in data1]
y1 = [data["label"] for data in data1]

X2 = [data["text"] for data in data2]
y2 = [data["label"] for data in data2]

X_official = [data["text"] for data in data_official]

In [3]:
def generate_probs(data):
  words = {}
  probs = {}

  for text in data:
    for i in range(len(text) - 1):
      if text[i] in words:
        words[text[i]] += 1
      else:
        words[text[i]] = 1

      if (text[i], text[i+1]) in probs:
        probs[(text[i], text[i+1])] += 1
      else:
        probs[(text[i], text[i+1])] = 1

  for (key, value) in probs.items():
    probs[key] /= words[key[0]]

  return probs

In [4]:
def prob_log(text, machine_probs, human_probs):
  temp = 0
  for i in range(len(text) - 1):
    if (text[i], text[i+1]) not in machine_probs:
      temp += 10
    else:
      temp += log10(machine_probs[(text[i], text[i+1])]) * -1

    if (text[i], text[i+1]) not in human_probs:
      temp -= 10
    else:
      temp -= log10(human_probs[(text[i], text[i+1])]) * -1

  return temp/len(text)

In [5]:
machine_X1 = [X1[i] for i in range(len(y1)) if y1[i] == 0]
human_X1 = [X1[i] for i in range(len(y1)) if y1[i] == 1]

machine_X2 = [X2[i] for i in range(len(y2)) if y2[i] == 0]
human_X2 = [X2[i] for i in range(len(y2)) if y2[i] == 1]

In [6]:
human_probs = generate_probs(human_X1 + human_X2)

machine_probs_list = []

len_part = int(len(machine_X2)/7)
for i in range(7):
  machine_probs_list.append(generate_probs(machine_X1 + machine_X2[i*len_part: (i+1) * len_part]))

In [7]:
y_official = []
for i in range(len(X_official)):
  temp = 0
  for j in range(len(machine_probs_list)):
    if prob_log(X_official[i], machine_probs_list[j], human_probs) >= 0:
      temp += 1
  if temp > 1:
    y_official.append(1)
  else:
    y_official.append(0)

In [8]:
id = range(4000)
answer = pd.DataFrame({'id': id, 'class': y_official})

answer.to_csv('submission.csv', index=False)